###Codigo e implementación https://github.com/Psychofun/Red-Neuronal-Numpy
Autor: PsyFun
##Preprocesamiento de los datos, para estandarizar ("gre","gpa") e individualizar la caracteristica categorica "rank"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd

admissions = pd.read_csv('/content/drive/MyDrive/7ma_matricula/Algoritmos/binary.csv')
print(admissions.describe())
print(admissions.shape)
print(admissions.head())

            admit         gre         gpa       rank
count  400.000000  400.000000  400.000000  400.00000
mean     0.317500  587.700000    3.389900    2.48500
std      0.466087  115.516536    0.380567    0.94446
min      0.000000  220.000000    2.260000    1.00000
25%      0.000000  520.000000    3.130000    2.00000
50%      0.000000  580.000000    3.395000    2.00000
75%      1.000000  660.000000    3.670000    3.00000
max      1.000000  800.000000    4.000000    4.00000
(400, 4)
   admit  gre   gpa  rank
0      0  380  3.61     3
1      1  660  3.67     3
2      1  800  4.00     1
3      1  640  3.19     4
4      0  520  2.93     4


#Conversion de una variable categorica "rank" a variables binarias "rank_1","rank_2","rank_3", "rank_4"

In [ ]:
data = pd.concat([admissions, pd.get_dummies(admissions['rank'], prefix='rank')], axis=1)
print(data.describe())
data = data.drop('rank', axis=1)
print(data.shape)
print(data.head())

            admit         gre         gpa       rank      rank_1      rank_2  \
count  400.000000  400.000000  400.000000  400.00000  400.000000  400.000000   
mean     0.317500  587.700000    3.389900    2.48500    0.152500    0.377500   
std      0.466087  115.516536    0.380567    0.94446    0.359955    0.485369   
min      0.000000  220.000000    2.260000    1.00000    0.000000    0.000000   
25%      0.000000  520.000000    3.130000    2.00000    0.000000    0.000000   
50%      0.000000  580.000000    3.395000    2.00000    0.000000    0.000000   
75%      1.000000  660.000000    3.670000    3.00000    0.000000    1.000000   
max      1.000000  800.000000    4.000000    4.00000    1.000000    1.000000   

           rank_3      rank_4  
count  400.000000  400.000000  
mean     0.302500    0.167500  
std      0.459916    0.373889  
min      0.000000    0.000000  
25%      0.000000    0.000000  
50%      0.000000    0.000000  
75%      1.000000    0.000000  
max      1.000000    1.

Estandarizacion de las variables ("gre","gpa")

In [ ]:
# Standarize features
for field in ['gre', 'gpa']:
    mean, std = data[field].mean(), data[field].std()
    data.loc[:,field] = (data[field]-mean)/std
print(data.shape)
print(data.describe())
print(data.head())

(400, 7)
            admit           gre           gpa      rank_1      rank_2  \
count  400.000000  4.000000e+02  4.000000e+02  400.000000  400.000000   
mean     0.317500 -3.907985e-16  2.198242e-16    0.152500    0.377500   
std      0.466087  1.000000e+00  1.000000e+00    0.359955    0.485369   
min      0.000000 -3.183094e+00 -2.968993e+00    0.000000    0.000000   
25%      0.000000 -5.860633e-01 -6.829288e-01    0.000000    0.000000   
50%      0.000000 -6.665712e-02  1.340106e-02    0.000000    0.000000   
75%      1.000000  6.258844e-01  7.360075e-01    0.000000    1.000000   
max      1.000000  1.837832e+00  1.603135e+00    1.000000    1.000000   

           rank_3      rank_4  
count  400.000000  400.000000  
mean     0.302500    0.167500  
std      0.459916    0.373889  
min      0.000000    0.000000  
25%      0.000000    0.000000  
50%      0.000000    0.000000  
75%      1.000000    0.000000  
max      1.000000    1.000000  
   admit       gre       gpa  rank_1  rank_2 

dividir el conjunto de datos para entrenamiento y evaluación del modelo

In [ ]:
print(data.index)

RangeIndex(start=0, stop=400, step=1)


In [ ]:
# Split off random 10% of the data for testing
np.random.seed(21)

sample = np.random.choice(data.index, size=int(len(data)*0.9), replace=False)
train_data, test_data = data.iloc[sample], data.drop(sample)
print(train_data.shape)
print(test_data.shape)

(360, 7)
(40, 7)


División de los datos en caracteristicas y etiquetas

In [ ]:
features, targets = train_data.drop('admit', axis=1), train_data['admit']
features_test, targets_test = test_data.drop('admit', axis=1), test_data['admit']

Función de Activación

In [ ]:
def sigmoide(x):
    return 1/(1 + np.exp(-x))

In [ ]:
# Hyperparameters
n_hidden = 2 # Número de unidades en la capa escondida
epochs = 300 # Número de iteraciones sobre el conjunto de entrenamiento
alpha = 0.01 # Taza de aprendizaje
np.random.seed(1)

ult_costo = None 

m,k = features.shape # Número de ejemplos de entrenamiento, número de dimensiones en los datos 
# Inicialización de los pesos
entrada_escondida = np.random.normal(scale = 1/k**0.5,size = (k,n_hidden))
escondida_salida = np.random.normal(scale = 1/k**0.5,size = n_hidden)
print(entrada_escondida.shape)
print(escondida_salida.shape)
# Entrenamiento
for e in range(epochs):
    # Variables para el gradiente
    gradiente_entrada_escondida = np.zeros(entrada_escondida.shape)
    gradiente_escondida_salida =  np.zeros(escondida_salida.shape)
    # Itera sobre el conjunto de entrenamiento
    for x,y in zip(features.values,targets):
        # Pasada hacia adelande (forward pass) or forward propagation
        z = sigmoide(np.matmul(x, entrada_escondida))
        y_ =sigmoide(np.matmul(escondida_salida,z)) # predicción 
        # Pasada hacia atrás (backward pass)
        salida_error = (y - y_) * y_ *(1- y_)
        escondida_error = np.dot(salida_error, escondida_salida) * z * (1 -z)

        gradiente_entrada_escondida += escondida_error * x[:,None]
        gradiente_escondida_salida += salida_error * z 
    # Actualiza los parámetros (pesos)
    entrada_escondida += alpha * gradiente_entrada_escondida / m 
    escondida_salida +=  alpha * gradiente_escondida_salida / m 

    if e % (epochs / 10 ) == 0:
        z = sigmoide(np.dot(features.values, entrada_escondida))
        y_ = sigmoide(np.dot(z, escondida_salida))

        # Función de costo
        costo = np.mean(( y_ - targets)**2 )

        if ult_costo  and ult_costo < costo:
            print("Costo de  entrenamiento: ", costo, " ADVERTENCIA -  Costo subiendo")
        else:
            print("Costo de entrenamiento: ", costo )
        
        ult_costo = costo 

#  Precisión en los datos de prueba 
z = sigmoide(np.dot(features_test, entrada_escondida))
y_ = sigmoide(np.dot(z, escondida_salida))

predicciones =  y_ > 0.5 
precision = np.mean(predicciones == targets_test)
print("Precisión: {:.3f}".format(precision))




(6, 2)
(2,)
Costo de entrenamiento:  0.2377095517475239
Costo de entrenamiento:  0.2372764787497153
Costo de entrenamiento:  0.23685185035695452
Costo de entrenamiento:  0.23643549079189158
Costo de entrenamiento:  0.23602722752904412
Costo de entrenamiento:  0.23562689126819572
Costo de entrenamiento:  0.23523431590566762
Costo de entrenamiento:  0.23484933850364967
Costo de entrenamiento:  0.23447179925776251
Costo de entrenamiento:  0.23410154146301557
Precisión: 0.650


#Cantidad de Épocas 

En este experimento buscamos encontrar una cantidad de épocas suficientes para entrenar nuestra red neuronal, sin entrar en un "over-fitting" 


In [ ]:
# Hyperparameters
n_hidden = 2 # Número de unidades en la capa escondida
epochs = 0 # Número de iteraciones sobre el conjunto de entrenamiento
alpha = 0.01 # Taza de aprendizaje

tol = 0.65  # La tolerancia de aprendizaje que definimos para nuestro modelo
precision = 0 # Precision con la que la red neuronal hace predicciónes

ult_costo = None 

m,k = features.shape # Número de ejemplos de entrenamiento, número de dimensiones en los datos 
# Inicialización de los pesos
entrada_escondida = np.random.normal(scale = 1/k**0.5,size = (k,n_hidden))
print(entrada_escondida)
escondida_salida = np.random.normal(scale = 1/k**0.5,size = n_hidden)
print(escondida_salida)
print(entrada_escondida.shape)
print(escondida_salida.shape)
# Entrenamiento
while precision < tol:
    epochs += 1
    # Variables para el gradiente
    gradiente_entrada_escondida = np.zeros(entrada_escondida.shape)
    gradiente_escondida_salida =  np.zeros(escondida_salida.shape)
    # Itera sobre el conjunto de entrenamiento
    for x,y in zip(features.values,targets):
        # Pasada hacia adelande (forward pass) or forward propagation
        z = sigmoide(np.matmul(x, entrada_escondida))
        y_ =sigmoide(np.matmul(escondida_salida,z)) # predicción 
        # Pasada hacia atrás (backward pass)
        salida_error = (y - y_) * y_ *(1- y_)
        escondida_error = np.dot(salida_error, escondida_salida) * z * (1 -z)

        gradiente_entrada_escondida += escondida_error * x[:,None]
        gradiente_escondida_salida += salida_error * z 
    # Actualiza los parámetros (pesos)
    entrada_escondida += alpha * gradiente_entrada_escondida / m 
    escondida_salida +=  alpha * gradiente_escondida_salida / m 

    #  Precisión en los datos de prueba 
    z = sigmoide(np.dot(features, entrada_escondida))
    y_ = sigmoide(np.dot(z, escondida_salida))

    predicciones =  y_ > 0.5 
    precision = np.mean(predicciones == targets)

    if epochs % (100) == 0:
        z = sigmoide(np.dot(features.values, entrada_escondida))
        y_ = sigmoide(np.dot(z, escondida_salida))

        # Función de costo
        costo = np.mean(( y_ - targets)**2 )

        if ult_costo  and ult_costo < costo:
            print("Costo de  entrenamiento: ", costo, " ADVERTENCIA -  Costo subiendo")
        else:
            print("Costo de entrenamiento: ", costo )
            print("Precisión: {:.3f}".format(precision))
            print("Numero de épocas: ", epochs)
        
        ult_costo = costo 

print("Modelo entrenado exitosamente !")
print("Precisión: {:.3f}".format(precision))
print("Numero de épocas: ", epochs)


[[-0.12306309  0.31317405]
 [-0.08645405 -0.02350285]
 [-0.76840206 -0.23939043]
 [ 0.33997091 -0.60476078]
 [ 0.17315152 -0.05567747]
 [-0.40734721 -0.05260147]]
[ 0.69885416 -0.59842473]
(6, 2)
(2,)
Costo de entrenamiento:  0.2604904269197874
Precisión: 0.378
Numero de épocas:  100
Costo de entrenamiento:  0.2580302037596837
Precisión: 0.397
Numero de épocas:  200
Costo de entrenamiento:  0.25569749959291455
Precisión: 0.442
Numero de épocas:  300
Costo de entrenamiento:  0.25348605729127915
Precisión: 0.467
Numero de épocas:  400
Costo de entrenamiento:  0.25138975866786617
Precisión: 0.486
Numero de épocas:  500
Costo de entrenamiento:  0.24940265262769856
Precisión: 0.536
Numero de épocas:  600
Costo de entrenamiento:  0.24751897711508844
Precisión: 0.542
Numero de épocas:  700
Costo de entrenamiento:  0.2457331756147034
Precisión: 0.556
Numero de épocas:  800
Costo de entrenamiento:  0.2440399089228966
Precisión: 0.597
Numero de épocas:  900
Costo de entrenamiento:  0.24243406285

#Cambio de función de activación

En este punto, se remplaza la función sigmoide por la tangente hiperbólica, para observar el comportamiento de la red neuronal

In [ ]:
# Hyperparameters
n_hidden = 2 # Número de unidades en la capa escondida
epochs = 300 # Número de iteraciones sobre el conjunto de entrenamiento
alpha = 0.01 # Taza de aprendizaje
np.random.seed(1)
ult_costo = None 

m,k = features.shape # Número de ejemplos de entrenamiento, número de dimensiones en los datos 
# Inicialización de los pesos
entrada_escondida = np.random.normal(scale = 1/k**0.5,size = (k,n_hidden))
escondida_salida = np.random.normal(scale = 1/k**0.5,size = n_hidden)
print(entrada_escondida.shape)
print(escondida_salida.shape)
# Entrenamiento
for e in range(epochs):
    # Variables para el gradiente
    gradiente_entrada_escondida = np.zeros(entrada_escondida.shape)
    gradiente_escondida_salida =  np.zeros(escondida_salida.shape)
    # Itera sobre el conjunto de entrenamiento
    for x,y in zip(features.values,targets):
        # Pasada hacia adelande (forward pass) or forward propagation
        z = np.tanh(np.matmul(x, entrada_escondida))
        y_ =np.tanh(np.matmul(escondida_salida,z)) # predicción 
        # Pasada hacia atrás (backward pass)
        salida_error = (y - y_) * (1 - np.power(y_,2))
        escondida_error = np.dot(salida_error, escondida_salida) * (1 - np.power(z,2))

        gradiente_entrada_escondida += escondida_error * x[:,None]
        gradiente_escondida_salida += salida_error * z 
    # Actualiza los parámetros (pesos)
    entrada_escondida += alpha * gradiente_entrada_escondida / m 
    escondida_salida +=  alpha * gradiente_escondida_salida / m 

    if e % (epochs / 10 ) == 0:
        z = np.tanh(np.dot(features.values, entrada_escondida))
        y_ = np.tanh(np.dot(z, escondida_salida))

        # Función de costo
        costo = np.mean(( y_ - targets)**2 )

        if ult_costo  and ult_costo < costo:
            print("Costo de  entrenamiento: ", costo, " ADVERTENCIA -  Costo subiendo")
        else:
            print("Costo de entrenamiento: ", costo )
        
        ult_costo = costo 

#  Precisión en los datos de prueba 
z = sigmoide(np.dot(features_test, entrada_escondida))
y_ = sigmoide(np.dot(z, escondida_salida))

predicciones =  y_ > 0.5 
precision = np.mean(predicciones == targets_test)
print("Precisión: {:.3f}".format(precision))



(6, 2)
(2,)
Costo de entrenamiento:  0.30965898521493956
Costo de entrenamiento:  0.2907808591951554
Costo de entrenamiento:  0.2769403554139474
Costo de entrenamiento:  0.26673328485052494
Costo de entrenamiento:  0.25912164031912954
Costo de entrenamiento:  0.2533579499725873
Costo de entrenamiento:  0.2489110846115626
Costo de entrenamiento:  0.24540572765162427
Costo de entrenamiento:  0.24257689979508013
Costo de entrenamiento:  0.24023718194086569
Precisión: 0.725
